In [2]:
import pandas as pd
import numpy as np
import random
from IPython.display import clear_output
import time
from transformers import BertTokenizer, TFBertModel

In [3]:
df1 = pd.read_csv('All_US_tweets.csv')

In [4]:
df1['TweetText'] = df1['TweetText'].astype(str)
df1 = df1[df1['TweetText'].str.split(" ").str.len() <= 29]
df1["TweetText"].str.split(" ").str.len().describe(percentiles=[0.01, 0.5, 0.99])

count    372389.000000
mean         11.709132
std           6.917127
min           1.000000
1%            2.000000
50%          10.000000
99%          28.000000
max          29.000000
Name: TweetText, dtype: float64

In [5]:
# Bert tools

b_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

def tokenize_tweet(tweet):
    return b_tokenizer(tweet, padding='max_length', truncation=True, return_tensors="tf")

#tokenized_tweets = df['tweet_text'].apply(tokenize_tweet)

def get_bert_embeddings(tokenized_tweet):
    outputs = bert_model(tokenized_tweet)
    return outputs.last_hidden_state[:, 0, :]

#embeddings = tokenized_tweets.apply(get_bert_embeddings)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [8]:
def time_return(start, end):
    total_time = end - start
    hours = total_time // 3600
    mins = total_time % 3600 // 60
    sec = total_time % 3600 % 60

    print("Execution took: " + str(hours) + ' hrs, ' + str(mins) + ' min, ' + str(np.round(sec,2)) + 'secs')

In [26]:
df2 = df1.head(500).copy()

In [27]:
df3 = df2['TweetText'].to_list()

In [16]:
start1 = time.time()

df2['TweetText'] = df2['TweetText'].apply(tokenize_tweet)
df2['TweetText'] = df2['TweetText'].apply(get_bert_embeddings)

end1 = time.time()


In [17]:
time_return(start1, end1)

Execution took: 0.0 hrs, 6.0 min, 4.76secs


In [28]:
start2 = time.time()

i = 0
imax = len(df3)
out = list()

while i < len(df3):

    text = df3[i]
    text = tokenize_tweet(text)
    text = get_bert_embeddings(text)
    df3[i] = text

    if i % 10 == 0:
        clear_output()
        print('Rows processed: ' + str(i) + '/' + str(imax))

    i+= 1

end2 = time.time()

Rows processed: 490/500


In [29]:
time_return(start2, end2)

Execution took: 0.0 hrs, 6.0 min, 13.25secs
